In [ ]:
!pip3 install -qU tf2onnx onnxruntime

# TODO: update training-v2 by main after merge of training-v2
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@training-v2

In [ ]:
import tensorflow as tf

In [ ]:
from wav2vec2 import Wav2Vec2ForCTC

model_id = "vasudevgupta/gsoc-wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_id)

In [ ]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/sample.wav

In [ ]:
import soundfile as sf
AUDIO_MAXLEN = 246000

speech, _ = sf.read("sample.wav")
speech = tf.constant(speech, dtype=tf.float32)[None, :AUDIO_MAXLEN]

In [ ]:
import tf2onnx
ONNX_PATH = "onnx-wav2vec2.onnx"

input_signature = (tf.TensorSpec((None, speech.shape[1]), tf.float32, name="speech"),)
_ = tf2onnx.convert.from_keras(model, input_signature=input_signature, output_path=ONNX_PATH)

In [ ]:
import onnxruntime as rt

sess = rt.InferenceSession(ONNX_PATH)
onnx_outputs = sess.run(None, {"speech": speech.numpy()})
onnx_outputs

In [ ]:
import time

start = time.time()
onnx_outputs = sess.run(None, {"speech": speech.numpy()})
print(onnx_outputs)
print("TIME TAKEN:", time.time() - start)

In [ ]:
start = time.time()
outputs = model(speech)
print(outputs)
print("TIME TAKEN:", time.time() - start)

In [ ]:
@tf.function(jit_compile=True)
def forward(speech):
    return model(speech)

In [ ]:
start = time.time()
outputs = forward(speech)
print(outputs)
print("TIME TAKEN:", time.time() - start)

In [ ]:
import numpy as np
np.allclose(onnx_outputs, outputs.numpy(), atol=1e-2)